The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module

## Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "For-Hire Vehicle Trip Records".

Download the data for January and February 2021.

Note that you need "For-Hire Vehicle Trip Records", not "High Volume For-Hire Vehicle Trip Records".

Read the data for January. How many records are there?

    1054112
    1154112
    1254112
    1354112


In [11]:
#!wget -P ./data https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-01.parquet
#!wget -P ./data https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-02.parquet

In [12]:
# temp cell of installing neccesary moduls
#!pip install fastparquet

In [13]:
import pandas as pd

In [14]:
data_01 = pd.read_parquet("./data/fhv_tripdata_2021-01.parquet")
data_02 = pd.read_parquet("./data/fhv_tripdata_2021-02.parquet")

In [15]:
data_01.sample(10)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
532398,B00310,2021-01-15 12:43:34,2021-01-15 13:29:25,NaN,247.0,None,B00310
540146,B02138,2021-01-15 14:37:55,2021-01-15 14:41:09,NaN,22.0,None,B02138
997467,B03266,2021-01-27 17:54:53,2021-01-27 18:04:25,223.0,7.0,None,B03266
53745,B02315,2021-01-03 01:06:21,2021-01-03 01:30:40,NaN,244.0,None,B02315
170464,B02550,2021-01-06 09:27:39,2021-01-06 09:29:54,NaN,174.0,None,B02550
592118,B01239,2021-01-17 07:38:33,2021-01-17 07:45:28,NaN,254.0,None,B02864
400383,B02782,2021-01-12 10:06:00,2021-01-12 10:42:00,NaN,NaN,None,B02782
862691,B01667,2021-01-24 12:18:55,2021-01-24 12:41:32,NaN,140.0,None,B01667
121170,B01559,2021-01-05 06:05:51,2021-01-05 06:17:51,NaN,NaN,None,B01559
10706,B01231,2021-01-01 11:40:54,2021-01-01 12:02:38,NaN,17.0,None,B02918
